# Librerias

In [ ]:
pip install librosa numpy pandas matplotlib tensorflow opencv-python scikit-learn seaborn

In [2]:
import os
import librosa
#import essentia.standard as es
import numpy as np
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import cv2
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input, concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, concatenate, BatchNormalization

# Funciones

In [ ]:
def extract_audio_features(file_path):
    try:
        # Cargar audio con librosa
        y, sr = librosa.load(file_path, mono=True)
        duration_ms = librosa.get_duration(y=y, sr=sr) * 1000  # Convertir a milisegundos

        # Extraer características con librosa
        features = {
            "duration_ms": duration_ms,
            "tempo": librosa.beat.tempo(y=y, sr=sr)[0],  # BPM
            "key": np.argmax(librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)),  # Aproximación de tonalidad
            "loudness": np.mean(librosa.feature.rms(y=y)),  # Root Mean Square Energy
            "energy": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),  # Energía del espectro
            "danceability": np.mean(librosa.feature.tempogram(y=y, sr=sr)),  # Relación con el ritmo
            "speechiness": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=1)),  # Proximidad a voz hablada
            "acousticness": np.mean(librosa.feature.spectral_flatness(y=y)),  # Nivel de acústica
            "instrumentalness": 1 - np.mean(librosa.feature.zero_crossing_rate(y=y)),  # Cantidad de transiciones en señal
            "liveness": np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),  # Sensación de "en vivo"
            "valence": np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),  # Percepción de alegría o tristeza
        }

        return features

    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)  # 13 coeficientes
    return np.mean(mfccs, axis=1)  # Se obtiene la media de cada coeficiente

#Creamos y almacenamos imagenes del espectrograma

In [ ]:
# 📂 Carpetas de entrada y salida
input_folders = {
    "Sobre": "/content/drive/MyDrive/canciones/Sobre",
    "Debajo": "/content/drive/MyDrive/canciones/debajo"
}
output_folder = "/content/drive/MyDrive/canciones/Espectrogramas"

# 📌 Asegurar que la carpeta de salida existe
os.makedirs(output_folder, exist_ok=True)

# 📌 Función para generar y guardar espectrogramas
def generar_espectrograma(input_folder, label):
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp3"):  # Filtrar solo archivos MP3
            file_path = os.path.join(input_folder, file_name)
            y, sr = librosa.load(file_path)  # Cargar audio

            # 📌 Generar el espectrograma
            S = librosa.feature.melspectrogram(y=y, sr=sr)
            S_db = librosa.power_to_db(S, ref=np.max)

            # 📌 Crear la figura del espectrograma
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f"Espectrograma - {file_name}")

            # 📌 Guardar la imagen con la categoría en el nombre
            output_path = os.path.join(output_folder, f"{label}_{file_name.replace('.mp3', '.png')}")
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            plt.close()

            print(f"Guardado: {output_path}")

# 📌 Generar espectrogramas para ambas carpetas
for label, folder in input_folders.items():
    generar_espectrograma(folder, label)




# Metricas de las canciones

In [ ]:
# Specify the folder containing the audio files
folder_path = "/content/drive/MyDrive/canciones/debajo"  # Replace with the actual folder path

audio_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)
               if file.endswith(('.mp3', '.wav', '.m4a'))]  # Add more extensions if needed

# Process each file and store in a DataFrame
data = []
for file in audio_files:
    features = extract_audio_features(file)
    if features:
        features["file_name"] = file  # Agregar nombre del archivo
        features["exito"] = "0"  # You might need to adjust this based on your file organization

        data.append(features)

# Crear DataFrame con Pandas
df = pd.DataFrame(data)

In [ ]:
# ... (extract_audio_features and extract_mfcc functions remain the same)

# Specify the folder containing the audio files for data2
folder_path2 = "/content/drive/MyDrive/canciones/Sobre"  # Replace with the actual folder path

# Get a list of all files in the folder for data2
audio_files2 = [os.path.join(folder_path2, file) for file in os.listdir(folder_path2)
                if file.endswith(('.mp3', '.wav', '.m4a'))]  # Add more extensions if needed

# Process each file and store in a DataFrame (data2)
data2 = []
for file in audio_files2:  # Use audio_files2 here
    features = extract_audio_features(file)
    if features:
        features["file_name"] = file  # Agregar nombre del archivo
        features["exito"] = "1"  # Change tipo to "more1million"

        data2.append(features)

# Crear DataFrame con Pandas (df2)
df2 = pd.DataFrame(data2)

In [ ]:
df.head()
df['file_name']=df['file_name'].str.split('/content/drive/MyDrive/canciones/debajo/').str[-1]
df.head()

In [ ]:
df2.head()
df2['file_name']=df2['file_name'].str.split('/content/drive/MyDrive/canciones/Sobre/').str[-1]
df2.head()

In [ ]:
df_combinado = pd.concat([df, df2], ignore_index=True)

In [ ]:
df_combinado.tail()

In [ ]:
# 📌 Guardar en Google Drive
csv_filename = "/content/drive/MyDrive/canciones/audio_features/audio_features.csv"
df_combinado.to_csv(csv_filename, index=False)

print(f"Archivo CSV guardado en: {csv_filename}")

# MODELO

In [ ]:
import os
import unicodedata

dir_path = r"C:\Users\santi\Documents\cursos\TalentoTech\canciones\Espectrogramas"
file_name = "Debajo_Lenny Tavárez, Ryan Castro - Ojos Chinos (Official Visualizer).png"

# Función para normalizar el nombre del archivo (eliminar espacios y tildes)
def normalize(name):
    name = name.replace(" ", "")  # Elimina espacios
    name = unicodedata.normalize("NFKD", name).encode("ASCII", "ignore").decode("utf-8")  # Quita tildes
    return name

# Normalizar el nombre objetivo
normalized_target = normalize(file_name)

# Buscar en el directorio
found = False
for file in os.listdir(dir_path):
    if normalize(file) == normalized_target:
        print(f"✅ Imagen encontrada: {file}")
        found = True
        break

if not found:
    print("❌ Imagen NO encontrada")



In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import unicodedata
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout,
                                     BatchNormalization, GlobalAveragePooling2D, Concatenate)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Rutas de archivos
CSV_PATH = r"./audio_features.csv"
IMAGE_FOLDER = r"./Espectrogramas/"
IMAGE_SIZE = (793, 373)

# Función para normalizar el nombre del archivo (eliminar espacios y tildes)
def normalize(name):
    name = name.replace(" ", "")  # Elimina espacios
    name = unicodedata.normalize("NFKD", name).encode("ASCII", "ignore").decode("utf-8")  # Quita tildes
    return name

# Cargar y procesar datos
def load_and_preprocess_data(csv_path, image_folder, image_size):
    df = pd.read_csv(csv_path)
    features = ['duration_ms', 'tempo', 'key', 'loudness', 'energy', 'danceability', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness', 'valence']
    y = df["exito"].values
    scaler = MinMaxScaler()

    X_images, valid_file_names = [], []
    for file_name in df["file_name"]:
        possible_names = [
            f"Sobre_{file_name.replace('.mp3', '.png')}",
            f"Debajo_{file_name.replace('.mp3', '.png')}"
        ]
        found_image = None
        for img_file in os.listdir(image_folder):
            if normalize(img_file) in [normalize(name) for name in possible_names]:
                found_image = img_file
                break
        
        if found_image:
            image_path = os.path.join(image_folder, found_image)
            try:
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, image_size)
                X_images.append(img)
                valid_file_names.append(file_name)
            except Exception:
                print(f"⚠️ Error al cargar la imagen: {image_path}")
        else:
            print(f"⚠️ Imagen no encontrada para: {file_name}")

    if not X_images:
        return None

    X_images = np.array(X_images) / 255.0
    X_images = X_images.reshape(-1, image_size[0], image_size[1], 1)
    df_filtered = df[df["file_name"].isin(valid_file_names)]
    X_numeric = scaler.fit_transform(df_filtered[features].values)
    y = df_filtered["exito"].values
    return X_numeric, X_images, y

# Construcción del modelo
def build_model(input_numeric_shape, image_size):
    l2_reg = l2(0.001)

    # Entrada de datos numéricos
    input_numeric = Input(shape=(input_numeric_shape,))
    redNProfunda = Dense(128, activation="relu", kernel_regularizer=l2_reg)(input_numeric)
    redNProfunda = BatchNormalization()(redNProfunda)
    redNProfunda = Dense(64, activation="relu", kernel_regularizer=l2_reg)(redNProfunda)

    # Entrada de imagen (CNN)
    input_image = Input(shape=(image_size[0], image_size[1], 1))
    redNConvolucional = Conv2D(64, (3,3), activation="relu", padding="same", kernel_regularizer=l2_reg)(input_image)
    redNConvolucional = BatchNormalization()(redNConvolucional)
    redNConvolucional = MaxPooling2D(pool_size=(2,2))(redNConvolucional)

    redNConvolucional = Conv2D(128, (3,3), activation="relu", padding="same", kernel_regularizer=l2_reg)(redNConvolucional)
    redNConvolucional = BatchNormalization()(redNConvolucional)
    redNConvolucional = MaxPooling2D(pool_size=(2,2))(redNConvolucional)

    redNConvolucional = Conv2D(256, (3,3), activation="relu", padding="same", kernel_regularizer=l2_reg)(redNConvolucional)
    redNConvolucional = BatchNormalization()(redNConvolucional)
    redNConvolucional = MaxPooling2D(pool_size=(2,2))(redNConvolucional)

    redNConvolucional = Conv2D(512, (3,3), activation="relu", padding="same", kernel_regularizer=l2_reg)(redNConvolucional)
    redNConvolucional = BatchNormalization()(redNConvolucional)
    redNConvolucional = MaxPooling2D(pool_size=(2,2))(redNConvolucional)

    # Aplanamos la salida de la CNN
    flattened_redNConvolucional = Flatten()(redNConvolucional)

    # Fusionamos la salida de la CNN con los datos numéricos
    redNFusionada = Concatenate()([flattened_redNConvolucional, redNProfunda])

    # Capas densas después de la fusión
    redNFusionada = Dense(128, activation="relu", kernel_regularizer=l2_reg)(redNFusionada)
    redNFusionada = Dropout(0.5)(redNFusionada)
    redNFusionada = Dense(64, activation="relu", kernel_regularizer=l2_reg)(redNFusionada)
    redNFusionada = Dropout(0.5)(redNFusionada)

    # Capa de salida
    output = Dense(1, activation="sigmoid")(redNFusionada)

    # Definir el modelo
    model = Model(inputs=[input_numeric, input_image], outputs=output)

    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss="binary_crossentropy", metrics=["accuracy"])
    
    return model
# Carga de datos
data = load_and_preprocess_data(CSV_PATH, IMAGE_FOLDER, IMAGE_SIZE)
if data:
    X_numeric, X_images, y = data
    X_train_num, X_test_num, X_train_img, X_test_img, y_train, y_test = train_test_split(
        X_numeric, X_images, y, test_size=0.2, random_state=42, stratify=y)

    model = build_model(X_train_num.shape[1], IMAGE_SIZE)
    early_stopping = EarlyStopping(monitor='val_loss', patience=80, restore_best_weights=True)
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

    history = model.fit(
        [X_train_num, X_train_img], y_train,
        validation_data=([X_test_num, X_test_img], y_test),
        epochs=150, batch_size=12, verbose=1,
        callbacks=[early_stopping] #reduce_lr]
    )

    y_pred_prob = model.predict([X_test_num, X_test_img])
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Evaluación con métricas adicionales
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)

    print(f"\n🔍 F1-Score: {f1:.4f}")
    print(f"🔍 AUC-ROC: {auc:.4f}")

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Éxito", "Éxito"], yticklabels=["No Éxito", "Éxito"])
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.title("Matriz de Confusión")
    plt.show()
else:
    print("Error en el procesamiento de datos.")



In [ ]:
# Diagramar la pérdida
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Pérdida en Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en Validación')
plt.title('Evolución de la Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import os
import shutil

# Ruta donde se guardará el modelo
MODEL_FOLDER = "./modelo_guardado/"
MODEL_PATH = os.path.join(MODEL_FOLDER, "modelo_prediccion_exito.h5")

# Eliminar la carpeta si ya existe
if os.path.exists(MODEL_FOLDER):
    shutil.rmtree(MODEL_FOLDER)

# Crear la carpeta nuevamente
os.makedirs(MODEL_FOLDER)

# Guardar el modelo en la nueva carpeta
model.save(MODEL_PATH)

print(f"✅ Modelo guardado en: {MODEL_PATH}")


# Prediccion

In [ ]:
# Cargar el modelo entrenado
import librosa

modelo = tf.keras.models.load_model("./modelo_entrenado_3/modelo_prediccion_exito.h5")  # Asegúrate de guardar el modelo entrenado

def extraer_caracteristicas_audio(archivo_mp3):
    try:
        # Cargar el audio con librosa
        y, sr = librosa.load(archivo_mp3, mono=True)
        duration_ms = librosa.get_duration(y=y, sr=sr) * 1000  # Convertir a milisegundos

        # Extraer características
        features = [
            duration_ms,
            librosa.beat.tempo(y=y, sr=sr)[0],  # BPM
            np.argmax(librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)),  # Aproximación de tonalidad
            np.mean(librosa.feature.rms(y=y)),  # Root Mean Square Energy (volumen)
            np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),  # Energía del espectro
            np.mean(librosa.feature.tempogram(y=y, sr=sr)),  # Relación con el ritmo
            np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=1)),  # Proximidad a voz hablada
            np.mean(librosa.feature.spectral_flatness(y=y)),  # Nivel de acústica
            1 - np.mean(librosa.feature.zero_crossing_rate(y=y)),  # Cantidad de transiciones en señal
            np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),  # Sensación de "en vivo"
            np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),  # Percepción de alegría o tristeza
        ]

        return np.array([features], dtype=np.float32)  # Forma correcta (1, 11)

    except Exception as e:
        print(f"Error procesando {archivo_mp3}: {e}")
        return None

def generar_espectrograma(archivo_mp3):
    """Genera un espectrograma de la canción y lo redNConvolucionalierte a un array de imagen."""
    y, sr = librosa.load(archivo_mp3, sr=None)
    plt.figure(figsize=(5, 5))
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    plt.axis('off')

    # Guardar el espectrograma temporalmente
    ruta_espectrograma = "temp_spec.png"
    plt.savefig(ruta_espectrograma, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Cargar el espectrograma como imagen y formatearlo
    img = cv2.imread(ruta_espectrograma, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (373, 793))  # Ajusta el tamaño según el modelo
    img = img / 255.0  # Normalización
    return np.expand_dims(img, axis=0)  # Añadir dimensión batch

def predecir_exito(archivo_mp3):
    """Realiza la predicción sobre un archivo MP3 dado."""
    # Extraer características numéricas
    X_num = extraer_caracteristicas_audio(archivo_mp3)

    # Generar espectrograma
    X_img = generar_espectrograma(archivo_mp3)

    # Hacer la predicción con el modelo
    prediccion = modelo.predict([X_num, X_img])

    # Interpretar la salida
    resultado = "Éxito" if prediccion[0][0] > 0.5 else "No éxito"
    print(prediccion)
    print(f"Predicción para {archivo_mp3}: {resultado} ({prediccion[0][0]:.4f})")

# Ejemplo de uso
archivo_prueba = "./prueba/Grupo Frontera ft. Morat - LOS DOS (Visualizer).mp3"  # Cambia esto por el archivo MP3 real
predecir_exito(archivo_prueba)

C:\Users\santi\AppData\Local\Temp\ipykernel_13904\3814532036.py:15: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  librosa.beat.tempo(y=y, sr=sr)[0],  # BPM


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step
[[0.]]
Predicción para ./prueba/Grupo Frontera ft. Morat - LOS DOS (Visualizer).mp3: No éxito (0.0000)
